In [3]:
import pandas as pd
from sqlalchemy import create_engine
import time

Running back data from week 6 of 2022. Used for development only. Change to loop to get all historic data for all positions and weeks. 

Game stats to scrape for "ground truth":
1. All positions
2. Weeks 1 through 18 inclusive
3. Years 2002 through 2022 inclusive
    * Not all years will be present in the projections data. Scraping large number of years to prevent having to come back to scrape later.

In [1]:
url = "https://www.fantasypros.com/nfl/stats/rb.php?year=2022&week=6&range=week"

In [4]:
df_list = pd.read_html(url)

In [5]:
data = df_list[0].copy(deep=True)

In [6]:
data.head()

Unnamed: 0_level_0        Unnamed: 1_level_0 RUSHING                         
                Rank                    Player     ATT  YDS   Y/A  LG 20+ TD   
0                  1  Rhamondre Stevenson (NE)      19   76   4.0  31   2  2  \
1                  2         Kenyan Drake (FA)      10  119  11.9  30   5  1   
2                  3        Deon Jackson (IND)      12   42   3.5  11   0  1   
3                  4         Breece Hall (NYJ)      20  116   5.8  34   2  1   
4                  5  Kenneth Walker III (SEA)      21   97   4.6  34   3  1   

  RECEIVING                 MISC                         
        REC TGT YDS  Y/R TD   FL  G  FPTS FPTS/G   ROST  
0         4   5  15  3.8  0    0  1  21.1   21.1  97.7%  
1         1   2   8  8.0  0    0  1  18.7   18.7  13.2%  
2        10  10  79  7.9  0    0  1  18.1   18.1   9.0%  
3         2   2   5  2.5  0    0  1  18.1   18.1  63.0%  
4         2   3  13  6.5  0    0  1  17.0   17.0  97.5%

Removing the multiindex by converting the columns to a numpy array, then joining the columns with their multiindex.

In [7]:
data.columns = ['_'.join(col) for col in data.columns.values]

In [8]:
data.head()

,Unnamed: 0_level_0_Rank,Unnamed: 1_level_0_Player,RUSHING_ATT,RUSHING_YDS,RUSHING_Y/A,RUSHING_LG,RUSHING_20+,RUSHING_TD,RECEIVING_REC,RECEIVING_TGT,RECEIVING_YDS,RECEIVING_Y/R,RECEIVING_TD,MISC_FL,MISC_G,MISC_FPTS,MISC_FPTS/G,MISC_ROST
0,1,Rhamondre Stevenson (NE),19,76,4.0,31,2,2,4,5,15,3.8,0,0,1,21.1,21.1,97.7%
1,2,Kenyan Drake (FA),10,119,11.9,30,5,1,1,2,8,8.0,0,0,1,18.7,18.7,13.2%
2,3,Deon Jackson (IND),12,42,3.5,11,0,1,10,10,79,7.9,0,0,1,18.1,18.1,9.0%
3,4,Breece Hall (NYJ),20,116,5.8,34,2,1,2,2,5,2.5,0,0,1,18.1,18.1,63.0%
4,5,Kenneth Walker III (SEA),21,97,4.6,34,3,1,2,3,13,6.5,0,0,1,17.0,17.0,97.5%


Renaming the Rand and Player columns to remove the "Unnamed" that is a result of the HTML

In [9]:
columns_renames = {data.columns[0]: 'RANK', data.columns[1]: 'PLAYER'}

data.rename(columns=columns_renames, inplace=True)

Splitting Player name from team and adding team to a new column

In [10]:
player_team = data.PLAYER.str.rsplit(" ", n=1, expand=True)

data.PLAYER = player_team[0]
data['TEAM'] = player_team[1].str.replace('\(|\)', '', regex=True)

In [11]:
data.head()

,RANK,PLAYER,RUSHING_ATT,RUSHING_YDS,RUSHING_Y/A,RUSHING_LG,RUSHING_20+,RUSHING_TD,RECEIVING_REC,RECEIVING_TGT,RECEIVING_YDS,RECEIVING_Y/R,RECEIVING_TD,MISC_FL,MISC_G,MISC_FPTS,MISC_FPTS/G,MISC_ROST,TEAM
0,1,Rhamondre Stevenson,19,76,4.0,31,2,2,4,5,15,3.8,0,0,1,21.1,21.1,97.7%,NE
1,2,Kenyan Drake,10,119,11.9,30,5,1,1,2,8,8.0,0,0,1,18.7,18.7,13.2%,FA
2,3,Deon Jackson,12,42,3.5,11,0,1,10,10,79,7.9,0,0,1,18.1,18.1,9.0%,IND
3,4,Breece Hall,20,116,5.8,34,2,1,2,2,5,2.5,0,0,1,18.1,18.1,63.0%,NYJ
4,5,Kenneth Walker III,21,97,4.6,34,3,1,2,3,13,6.5,0,0,1,17.0,17.0,97.5%,SEA


Adding Year, Week, and Position

In [12]:
data['YEAR'] = 2022
data['POS'] = "RB"
data['WEEK'] = 6

In [13]:
data.head()

,RANK,PLAYER,RUSHING_ATT,RUSHING_YDS,RUSHING_Y/A,RUSHING_LG,RUSHING_20+,RUSHING_TD,RECEIVING_REC,RECEIVING_TGT,...,RECEIVING_TD,MISC_FL,MISC_G,MISC_FPTS,MISC_FPTS/G,MISC_ROST,TEAM,YEAR,POS,WEEK
0,1,Rhamondre Stevenson,19,76,4.0,31,2,2,4,5,...,0,0,1,21.1,21.1,97.7%,NE,2022,RB,6
1,2,Kenyan Drake,10,119,11.9,30,5,1,1,2,...,0,0,1,18.7,18.7,13.2%,FA,2022,RB,6
2,3,Deon Jackson,12,42,3.5,11,0,1,10,10,...,0,0,1,18.1,18.1,9.0%,IND,2022,RB,6
3,4,Breece Hall,20,116,5.8,34,2,1,2,2,...,0,0,1,18.1,18.1,63.0%,NYJ,2022,RB,6
4,5,Kenneth Walker III,21,97,4.6,34,3,1,2,3,...,0,0,1,17.0,17.0,97.5%,SEA,2022,RB,6


In [199]:
for column in data.columns:
    #if data[column].dtype ==
    print(type(data[column].dtype))

<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[object_]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[float64]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[float64]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[float64]'>
<class 'numpy.dtype[float64]'>
<class 'numpy.dtype[object_]'>
<class 'numpy.dtype[object_]'>
<class 'numpy.dtype[int64]'>
<class 'numpy.dtype[object_]'>
<class 'numpy.dtype[int64]'>


In [197]:
data.columns

Index(['RANK', 'PLAYER', 'RUSHING_ATT', 'RUSHING_YDS', 'RUSHING_Y/A',
       'RUSHING_LG', 'RUSHING_20+', 'RUSHING_TD', 'RECEIVING_REC',
       'RECEIVING_TGT', 'RECEIVING_YDS', 'RECEIVING_Y/R', 'RECEIVING_TD',
       'MISC_FL', 'MISC_G', 'MISC_FPTS', 'MISC_FPTS/G', 'MISC_ROST', 'TEAM',
       'YEAR', 'POS', 'WEEK'],
      dtype='object')

In [16]:
engine = create_engine('sqlite:///data/historic_data.db')
sqlite_connection = engine.connect()

In [17]:
sqlite_table = "Game_stats"

data.to_sql(sqlite_table, sqlite_connection, if_exists='append')

sqlite_connection.close()

In [99]:

engine.connect()

In [ ]:
positions = ['qb', 'rb', 'te', 'k', 'dst', 'dl', 'lb', 'db']
years = list(range(2002, 2023))
weeks = list(range(1,19))

positions = ['qb', 'rb', 'te', 'k', 'dst', 'dl', 'lb', 'db']
years = list(range(2002, 2023))
weeks = list(range(1,19))

template_url = f"https://www.fantasypros.com/nfl/stats/{position}.php?year={year}&week={week}&range=week"

urls = []

for position in positions:
    for year in years:
        for week in weeks:
            urls.append(template_url)

In [200]:
positions = ['qb', 'rb']
years = list(range(2002, 2004))
weeks = list(range(1,3))

urls = []

for position in positions:
    for year in years:
        for week in weeks:
            template_url = f"https://www.fantasypros.com/nfl/stats/{position}.php?year={year}&week={week}&range=week"
            
            urls.append([template_url, position, year, week])
            
print(urls)

[['https://www.fantasypros.com/nfl/stats/qb.php?year=2002&week=1&range=week', 'qb', 2002, 1], ['https://www.fantasypros.com/nfl/stats/qb.php?year=2002&week=2&range=week', 'qb', 2002, 2], ['https://www.fantasypros.com/nfl/stats/qb.php?year=2003&week=1&range=week', 'qb', 2003, 1], ['https://www.fantasypros.com/nfl/stats/qb.php?year=2003&week=2&range=week', 'qb', 2003, 2], ['https://www.fantasypros.com/nfl/stats/rb.php?year=2002&week=1&range=week', 'rb', 2002, 1], ['https://www.fantasypros.com/nfl/stats/rb.php?year=2002&week=2&range=week', 'rb', 2002, 2], ['https://www.fantasypros.com/nfl/stats/rb.php?year=2003&week=1&range=week', 'rb', 2003, 1], ['https://www.fantasypros.com/nfl/stats/rb.php?year=2003&week=2&range=week', 'rb', 2003, 2]]


In [203]:
for url in urls:
    print(url[0])
    time.sleep(2)

https://www.fantasypros.com/nfl/stats/qb.php?year=2002&week=1&range=week
https://www.fantasypros.com/nfl/stats/qb.php?year=2002&week=2&range=week
https://www.fantasypros.com/nfl/stats/qb.php?year=2003&week=1&range=week
https://www.fantasypros.com/nfl/stats/qb.php?year=2003&week=2&range=week


KeyboardInterrupt: 

In [205]:
with open('log.txt') as log:
    print("scraping", file=log)

FileNotFoundError: [Errno 2] No such file or directory: 'log.txt'

In [207]:
data.columns = ['_'.join(col) for col in data.columns.values]

In [208]:
data.head()

,R_A_N_K,P_L_A_Y_E_R,R_U_S_H_I_N_G___A_T_T,R_U_S_H_I_N_G___Y_D_S,R_U_S_H_I_N_G___Y_/_A,R_U_S_H_I_N_G___L_G,R_U_S_H_I_N_G___2_0_+,R_U_S_H_I_N_G___T_D,R_E_C_E_I_V_I_N_G___R_E_C,R_E_C_E_I_V_I_N_G___T_G_T,...,R_E_C_E_I_V_I_N_G___T_D,M_I_S_C___F_L,M_I_S_C___G,M_I_S_C___F_P_T_S,M_I_S_C___F_P_T_S_/_G,M_I_S_C___R_O_S_T,T_E_A_M,Y_E_A_R,P_O_S,W_E_E_K
0,1,Rhamondre Stevenson,19,76,4.0,31,2,2,4,5,...,0,0,1,21.1,21.1,97.7%,NE,2022,RB,6
1,2,Kenyan Drake,10,119,11.9,30,5,1,1,2,...,0,0,1,18.7,18.7,13.2%,FA,2022,RB,6
2,3,Deon Jackson,12,42,3.5,11,0,1,10,10,...,0,0,1,18.1,18.1,9.0%,IND,2022,RB,6
3,4,Breece Hall,20,116,5.8,34,2,1,2,2,...,0,0,1,18.1,18.1,63.0%,NYJ,2022,RB,6
4,5,Kenneth Walker III,21,97,4.6,34,3,1,2,3,...,0,0,1,17.0,17.0,97.5%,SEA,2022,RB,6


In [209]:
url = 'https://www.fantasypros.com/nfl/stats/db.php?year=2022&week=5&range=week'

db_df_list = pd.read_html(url)

In [211]:
db_df_list

[     Rank                            Player  TACKLE  ASSIST  SACK  PD  INT   
 0       1              DeShon Elliott (MIA)      10       2   0.0   1    1  \
 1       2         Antoine Winfield Jr. (TB)       7       1   1.0   1    0   
 2       3        C.J. Gardner-Johnson (DET)      10       0   0.0   1    1   
 3       4            Byron Murphy Jr. (MIN)       7       2   0.5   1    0   
 4       5                  Kaiir Elam (BUF)       7       3   0.0   1    1   
 ..    ...                               ...     ...     ...   ...  ..  ...   
 483   484                Bopete Keyes (BAL)       0       0   0.0   0    0   
 484   485                   Shaun Wade (NE)       0       0   0.0   0    0   
 485   486                  Tino Ellis (MIA)       0       0   0.0   0    0   
 486   487                Elijah Riley (PIT)       0       0   0.0   0    0   
 487   488  Stantley Thomas-Oliver III (CAR)       0       0   0.0   0    0   
 
      FF  FR  DEF TD  G  FPTS  FPTS/G   ROST  
 0 

In [212]:
pwd

'/home'

In [215]:
sqlite_connection.close()

In [18]:
url = "https://www.fantasypros.com/nfl/stats/k.php?year=2002&week=1&range=week"

In [19]:
scraped_data = pd.read_html(url)[0]

In [20]:
scraped_data.head()

,Rank,Player,FG,FGA,PCT,LG,1-19,20-29,30-39,40-49,50+,XPT,XPA,G,FPTS,FPTS/G,ROST
0,1,Olindo Mare (FA),0,0,0.0,0,0,0,0,0,0,7,7,1,7.0,7.0,0.0%
1,2,Mike Hollis (FA),1,2,50.0,52,0,0,0,0,0,4,4,1,4.0,4.0,0.0%
2,3,Steve Christie (FA),2,2,100.0,28,0,0,0,0,0,4,4,1,4.0,4.0,0.0%
3,4,Morten Andersen (FA),2,2,100.0,33,0,0,0,0,0,4,4,1,4.0,4.0,0.0%
4,5,Brett Conway (FA),1,1,100.0,35,0,0,0,0,0,4,4,1,4.0,4.0,0.0%


In [23]:
scraped_data.columns = scraped_data.columns.str.upper()

In [24]:
scraped_data.head()

,RANK,PLAYER,FG,FGA,PCT,LG,1-19,20-29,30-39,40-49,50+,XPT,XPA,G,FPTS,FPTS/G,ROST
0,1,Olindo Mare (FA),0,0,0.0,0,0,0,0,0,0,7,7,1,7.0,7.0,0.0%
1,2,Mike Hollis (FA),1,2,50.0,52,0,0,0,0,0,4,4,1,4.0,4.0,0.0%
2,3,Steve Christie (FA),2,2,100.0,28,0,0,0,0,0,4,4,1,4.0,4.0,0.0%
3,4,Morten Andersen (FA),2,2,100.0,33,0,0,0,0,0,4,4,1,4.0,4.0,0.0%
4,5,Brett Conway (FA),1,1,100.0,35,0,0,0,0,0,4,4,1,4.0,4.0,0.0%
